In [2]:
# Creates new business-level conceptual entities into the DB from an excel sheet
import sqlite3
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append('../')
from user_packages import hashing

In [3]:
# Set script variables
source_file_name = 'Org_Dictionary(Systems).csv'
source_file_path = '../imports'

record_source = 'SystemCSV'

In [4]:
conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

In [12]:
# read table to df
df = pd.read_csv(os.path.join(source_file_path, source_file_name))
df = df.replace({np.nan: None})

#df

In [6]:
# Create additional fields
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()



In [13]:
# Hash fields


df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'SystemHashKey'
  , columns = ['SystemKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['SystemName','Description','isDeleted']
)


#df

In [8]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CreateSystem")
conn.commit()
df.to_sql('stg_Py_CreateSystem', conn, if_exists='append', index=False)

19

In [9]:
# Write to the Hub (System)
sql_query = """
INSERT INTO rv_h_System
(
    SystemHashKey
  , LoadDate
  , RecordSource
  , SystemKeyPhrase
)
SELECT DISTINCT
    SystemHashKey
  , LoadDate
  , RecordSource
  , SystemKeyPhrase
FROM
  stg_Py_CreateSystem
WHERE
  SystemHashKey NOT IN (SELECT SystemHashKey FROM rv_h_System)
""";
conn.execute(sql_query)
conn.commit()

In [11]:
# Write to the Satellite (System)
sql_query = """
INSERT INTO rv_s_System
(
    SystemHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , SystemName
  , Description
  , isDeleted
)
SELECT DISTINCT
    stg.SystemHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.SystemName
  , stg.Description
  , stg.isDeleted
FROM
  stg_Py_CreateSystem AS stg
  LEFT OUTER JOIN rv_s_System AS sat ON (
    stg.SystemHashKey = sat.SystemHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_System AS z
      WHERE z.SystemHashKey = sat.SystemHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()